In [13]:
%%capture
%run final_chars_classification.ipynb

In [14]:
# from collections import defaultdict
# import os
# d = defaultdict(int)
# co = 0
# directory = './symbols/'
# for x in sorted(os.listdir(directory)):
#     if not x.startswith('.') and not d[x]:
#         d[x] = co
#         co+=1
# d

In [15]:
import joblib
import cv2
import numpy as np
from commonfunctions import * 
# from ipynb.fs.full.letters_extraction import *
# from ipynb.fs.full.final_chars_classification import *
# save the model to disk
filename = './saved_models/finalized_model.sav'
model = joblib.load(filename)
# winSize = (32,32)
winSize = (16,16)
blockSize = (8,8)
blockStride = (8,8)
cellSize = (8,8)
nbins = 20
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)

def predict_image(image, verbose= False):
    """
    This function takes in an image and returns the predicted letters
    in general, it takes in an image and returns a list of letters as Chars
    """
    filename = './saved_models/finalized_model.sav'
    model = joblib.load(filename)
    if(model == None):
        raise Exception("Model not found")
    
    
    letters_image = image
    try:
        letters_image= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except:
        pass
    letters_res = np.array(get_letters(letters_image, single_letter=False, verbose= False))
    if verbose:
        show_images([letters_image])
        show_images(letters_res)
    # cf.show_images([letters_res[0], get_letters(letters, single_letter=False)[0]])
    # letters_res = letters_res.reshape(len(letters_res), -1)
    letters_res = np.array([hog.compute(image) for image in letters_res])
    results = classify(letters_res)
    # print(results)
    
    return results



In [16]:

def get_table(img):
    results = predict_image(img, verbose= False)
    results[1] = '=' # the second element is always the equal sign
    s = "".join(results)
    splited_first , splited_second = s.split('=')

    all_valid_literals = ['A','B', 'C', 'F']
    def delete_invalid(variable):
        # all_valid_literals = ['A','B', 'C', 'F']
        # all_valid_chars.remove(splited_first.upper().strip())
        return variable in all_valid_literals
        

    final_str = ""
    all_valid_chars = all_valid_literals 
    for ind, char in enumerate(splited_second):
        if(char == '+'):
            char = 'or'
            
        let = lambda x: (x in all_valid_chars)
        
        # the prev was a literal and the current is a literal or ( 
        # or the prev was ) and the current is a literal or (
        # 
        if( ind != 0 and
            (
                (let(splited_second[ind-1]) and (let(char) or char == '(')) or 
                (splited_second[ind-1] == ')' and (let(char) or char == '(') )
            )
        ):
            final_str += "and "
        prev_was_literal = char in all_valid_literals
        final_str += (char + " ")

    print(final_str)

    from  ttg import Truths
    filters_chars = list(set(filter(delete_invalid, splited_second)))
    try:
        table = Truths( filters_chars , [final_str])
        return table
    except:
        return None


img =  cv2.imread(r".\test_images\classification\test4.png")
img =  cv2.imread(r".\test_images\classification\brackets.png")
print(get_table(img))

( A or B ) and ( C or A ) 
+-----+-----+-----+------------------------------+
|  A  |  C  |  B  |  ( A or B ) and ( C or A )   |
|-----+-----+-----+------------------------------|
|  1  |  1  |  1  |              1               |
|  1  |  1  |  0  |              1               |
|  1  |  0  |  1  |              1               |
|  1  |  0  |  0  |              1               |
|  0  |  1  |  1  |              1               |
|  0  |  1  |  0  |              0               |
|  0  |  0  |  1  |              0               |
|  0  |  0  |  0  |              0               |
+-----+-----+-----+------------------------------+
